In [1]:
import pandas as pd
import numpy as np
import torchvision as tv
import jieba
from gensim import models
from gensim.models import Word2Vec
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
import torch
show = ToPILImage() # 可以把Tensor转成Image，方便可视化
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('../../../data/df_sen_sub/train.csv')
data

,content_id,content,subject,sentiment_value,sentiment_word
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高
2,QmqJ2AvM5GplaRyz,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低
3,KMT1gFJiU4NWrVDn,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,价格,-1,有钱任性
4,nVIlGd5yMmc37t1o,17价格忒高，估计也就是14-15左右。,价格,-1,高
5,TVciHBPL5XmUxMEd,我开始就是荣放2.5 森林人2.5二选一 荣放主要是底盘质感不行 太硬 其次是...,价格,1,便宜
6,4lb9TsO5rjqUy1Bu,唉，这货的价格死硬死硬的，低配版优惠1万据说已经罕有了。,价格,-1,死硬
7,z98xV7MphZIF4EvO,价格的话只能说一般般吧，太仓前段时间定的比你便宜！,价格,0,一般
8,hoWrMnGTUvdsK3Dm,听过，价格太贵，但一直念念不忘,价格,-1,贵
9,r1WHgbI0ZKqMdELy,恭喜恭喜，这个优惠不错哦！,价格,1,优惠不错


In [3]:
'''
data = pd.read_csv('../../data/df_sen_sub/train.csv')
len(data['content_id'].unique()), len(list(data['content_id']))
content_ids = data['content_id'].unique()
subjs = data['subject'].unique()
df = {}
text, subs, sentis = [], [], []
for i in range(len(content_ids)):
    if i % 100 == 0:print(i)
    dd = data[data['content_id'] == content_ids[i]]
    text.append(dd.loc[dd.index.tolist()[0], 'content'])
    temp_subs, temp_senti = [], []
    for sub in subjs:
        temp_subs.append(sub)
        if len(dd[dd['subject'] == sub]) != 0:
            temp_senti.append(str(int(dd[dd['subject'] == sub]['sentiment_value'])))
        else:
            temp_senti.append('2')
    subs.append(','.join(temp_subs))
    sentis.append(','.join(temp_senti))
df['content'] = text
df['subjects'] = subs
df['sentiments'] = sentis
df = pd.DataFrame(df)
df.to_csv('../../data/df_sen_sub/merge_train.csv', index = False)
#data = pd.read_csv('../../data/df_sen_sub/test_public.csv')
'''

"\ndata = pd.read_csv('../../data/df_sen_sub/train.csv')\nlen(data['content_id'].unique()), len(list(data['content_id']))\ncontent_ids = data['content_id'].unique()\nsubjs = data['subject'].unique()\ndf = {}\ntext, subs, sentis = [], [], []\nfor i in range(len(content_ids)):\n    if i % 100 == 0:print(i)\n    dd = data[data['content_id'] == content_ids[i]]\n    text.append(dd.loc[dd.index.tolist()[0], 'content'])\n    temp_subs, temp_senti = [], []\n    for sub in subjs:\n        temp_subs.append(sub)\n        if len(dd[dd['subject'] == sub]) != 0:\n            temp_senti.append(str(int(dd[dd['subject'] == sub]['sentiment_value'])))\n        else:\n            temp_senti.append('2')\n    subs.append(','.join(temp_subs))\n    sentis.append(','.join(temp_senti))\ndf['content'] = text\ndf['subjects'] = subs\ndf['sentiments'] = sentis\ndf = pd.DataFrame(df)\ndf.to_csv('../../data/df_sen_sub/merge_train.csv', index = False)\n#data = pd.read_csv('../../data/df_sen_sub/test_public.csv')\n"

In [4]:
data = pd.read_csv('../../../data/df_sen_sub/merge_train.csv')
data = data.fillna('')
subs = list(data['subjects'])
sentiments = list(data['sentiments'])
content = list(data['content'])
test_data = pd.read_csv('../../../data/df_sen_sub/test_public.csv')
test_data = test_data.fillna('')
test_content = list(test_data['content'])
data = pd.read_csv('../../../data/df_sen_sub/train.csv')
data = data.fillna('')
senti_words = list(data['sentiment_word'])

In [5]:
with open('../../../data/df_sen_sub/add_text.txt','w') as f:
    f.write('\n'.join(list(content) + list(test_content)  + list(subs) + senti_words))
# 迭代器，使用jieba将句子进行分词
stopwords = open('../stopwords.txt', 'r').read().split('\n')
class Sentences(object):# 这个类可以根据实际情况重写，我已经将所有的文章进行分句，并整合到了一个文件里面
    def __init__(self, dirname):
        self.dirname = dirname # 句子所在文件，没句句子占一行
        #jieba.load_userdict("wordBase.txt") # 加载词库

    def __iter__(self):
        #for fname in os.listdir(self.dirname):
        for line in open(self.dirname):
                line = line.replace('\n', '')
                res = []
                for word in list(jieba.cut(line)):
                    if word in stopwords:
                        continue
                    res.append(word)
                yield res#list(jieba.cut(line))

sentences = []
def train_word2vec(folder_path, size=100):
    global sentences
    sentences = Sentences(folder_path) #生成分词后的句子，是一个二维数组

    # size是词向量长度
    # worker是线程数量，建议与物理线程数量一致
    # min_count是指出现次数小于一定程度，就忽略，0表示不忽略
    #model = Word2Vec(sentences, size=size, workers=8, min_count=0)

    # 训练结束就将模型保存起来
    #model.save("../../data/df_sen_sub/add_word2vec_model")

# 生成50维度的词向量模型
train_word2vec("../../../data/df_sen_sub/add_text.txt",100)

# 测试训练好的词向量模型，使用model[keyWord]即可获取keyword这个词的词向量
model = Word2Vec.load("../../../data/df_sen_sub/add_word2vec_model")
sentences = list(sentences)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.299 seconds.
Prefix dict has been built succesfully.


In [6]:
len(sentences)

28890

In [9]:
from collections import Counter
data = pd.read_csv('../../../data/df_sen_sub/train.csv')
sub_uniq = data['subject'].unique()
data = data[pd.notnull(data['sentiment_word'])]
senti_word_set = {}
sum_ = 0
for key in sub_uniq:
    co = 0
    senti_word_set[key] = {}
    #dd = list(data[data['subject'] == key]['sentiment_word'])
    ddd = data[data['subject'] == key]
    for kk in [-1, 0, 1]:
        dd = list(ddd[ddd['sentiment_value'] == kk]['sentiment_word'])
        ss = []
        for phras in dd:
            words = list(jieba.cut(phras))
            for word in words:
                word = word.replace(' ', '')
                if word == '' or word in stopwords:continue
                ss.append(word)
                co += 1
            counter = Counter(ss)
            sort = sorted(counter.items(), key=lambda item: item[1], reverse = True)
            for mm in counter:
                if mm not in senti_word_set[key]:
                    senti_word_set[key][mm] = kk
                elif senti_word_set[key][mm] == kk:
                    continue
                else:
                    senti_word_set[key][mm] = 0
        sum_ += co
print(sum_)
senti_word_set

6431


{'价格': {'高': 0,
  '有钱': -1,
  '任性': -1,
  '死硬': -1,
  '贵': 0,
  '优惠': 0,
  '太小': -1,
  '套路': -1,
  '太贵': -1,
  '不值': -1,
  '高昂': -1,
  '下不来': -1,
  '不会': 0,
  '便宜': 0,
  '放弃': -1,
  '平易近人': -1,
  '价格': 0,
  '老贵': -1,
  '没有': 0,
  '保证': -1,
  '好多': -1,
  '更': -1,
  '合适': 0,
  '那点': -1,
  '贵太多': -1,
  '挺高': -1,
  '不菲': -1,
  '一点': 0,
  '奇高': -1,
  '不好': -1,
  '小贵': 0,
  '过高': -1,
  '售价': -1,
  '明显': -1,
  '高于': -1,
  '竞品': -1,
  '优势': -1,
  '虚高': -1,
  '不高': 0,
  '性价比': 0,
  '不算': -1,
  '单价': -1,
  '没': 0,
  '降价': -1,
  '不要': 0,
  '想': -1,
  '保值': -1,
  '率': -1,
  '大众': -1,
  '好': 0,
  '略贵': -1,
  '保养': -1,
  '不用': -1,
  '考虑': -1,
  '不靠': -1,
  '谱': -1,
  '没法': -1,
  '咬咬牙': -1,
  '不合算': -1,
  '影响': 0,
  '亲民点': 0,
  '差不多': 0,
  '差距': 0,
  '知道': 0,
  '合理': 0,
  '就行': 0,
  '行情': 0,
  '价': 0,
  '正常': 0,
  '几百': 0,
  '参差不齐': 0,
  '行': 0,
  '买': 0,
  '价差': 0,
  '很大': 0,
  '低': 0,
  '不能': 0,
  '在意': 0,
  '还行': 0,
  '不错': 1,
  '幅度': 1,
  '不小': 1,
  '后悔': 1,
  '值': 1,
  '愉快': 1,
  '不够': 1,
  '比较'

In [10]:
x_vecs = []
x_test_vecs = []  
ind = 0
word2ind = {}
pretrained = []
senti_sen = []
senti_test_sen = []
for i in range(len(test_content) + len(content)):
    if i < len(content):
        temp = []
        t_s = []
        for j in range(len(sentences[i])):
            if sentences[i][j] not in word2ind:
                pretrained.append(model[sentences[i][j]])
                word2ind[sentences[i][j]] = ind
                ind += 1
            t_s.append(sentences[i][j])
            temp.append(model[sentences[i][j]])
        while len(temp) < 50:
            temp.append([0.0] * 100)
        if len(temp) > 50:
            temp = temp[:50]
        x_vecs.append(temp)
        senti_sen.append(t_s[:50])
    else:
        temp = []
        t_s = []
        for j in range(len(sentences[i])):
            if sentences[i][j] not in word2ind:
                pretrained.append(model[sentences[i][j]])
                word2ind[sentences[i][j]] = ind
                ind += 1
            t_s.append(sentences[i][j])
            temp.append(model[sentences[i][j]])
        while len(temp) < 50:
            temp.append([0.0] * 100)
        if len(temp) > 50:
            temp = temp[:50]
        x_test_vecs.append(temp)
        senti_test_sen.append(t_s[:50])     
x_vecs = np.array(x_vecs)
x_test_vecs = np.array(x_test_vecs)
print(x_vecs.shape, x_test_vecs.shape)

/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(8290, 50, 100) (2364, 50, 100)


In [11]:
x_sent_vecs = {}
x_test_sent_vecs = {}
for key in senti_word_set:
    x_sent_vecs[key] = []
    x_test_sent_vecs[key] = []
    for i in range(len(content) + len(test_content)):
        if i < len(content):
            t_sent = []
            for j in range(len(sentences[i])):
                if sentences[i][j] in senti_word_set[key]:
                    t_sent.append(senti_word_set[key][sentences[i][j]])
                else:
                    t_sent.append(0) 
            while len(t_sent) < 50:
                t_sent.append(0)
            if len(t_sent) > 50:
                t_sent = t_sent[:50]
            x_sent_vecs[key].append(t_sent)
        else:
            t_sent = []
            for j in range(len(sentences[i])):
                if sentences[i][j] in senti_word_set[key]:
                    t_sent.append(senti_word_set[key][sentences[i][j]])
                else:
                    t_sent.append(0) 
            while len(t_sent) < 50:
                t_sent.append(0)
            if len(t_sent) > 50:
                t_sent = t_sent[:50]
            x_test_sent_vecs[key].append(t_sent)    

In [12]:
#va, content_vecs, y
pretrained.append([0.0] * 100)
word2ind['null'] = 17387
ind2word = {}
for key in word2ind:
    ind2word[word2ind[key]] = key
map_ = ['-1', '0', '1', '2']
sub_set = {}
for i in range(len(subs)):
    subjs = subs[i].split(',')
    sentis = sentiments[i].split(',')
    for j in range(len(subjs)):
        if subjs[j] not in sub_set:
            sub_set[subjs[j]] = []
        if sentis[j] == '2':
            sub_set[subjs[j]].append(0)
        else:
            sub_set[subjs[j]].append(1)
        #sub_set[subjs[j]].append(map_.index(sentis[j]))

In [13]:
pretrained = np.array(pretrained)
han_x = []
for i in range(len(senti_sen)):
    temp = []
    for j in range(len(senti_sen[i])):
        temp.append(word2ind[senti_sen[i][j]])
    while len(temp) < 50:
        temp.append(17387)

    temp = temp[:50]

    han_x.append(temp)
han_x = np.array(han_x)

han_x_test = []
for i in range(len(senti_test_sen)):
    temp = []
    for j in range(len(senti_test_sen[i])):
        temp.append(word2ind[senti_test_sen[i][j]])
    while len(temp) < 50:
        temp.append(17387)

    temp = temp[:50]

    han_x_test.append(temp)
han_x = np.array(han_x)
han_x_test = np.array(han_x_test)

In [14]:

'''
class Net(nn.Module):
    def __init__(self, pretrained, vocab_size):
        super(Net, self).__init__()
        self.bilstm = nn.LSTM(input_size = 100, hidden_size = 100, bidirectional = True) 
        #self.fc1   = nn.Linear(3750, 50) 
        #self.fc2   = nn.Linear(200, 50)
        #self.fc3   = nn.Linear(200, 50)
        #self.fc4   = nn.Linear(100, 3)
        self.fctest   = nn.Linear(200, 3)
        self.embeddings = nn.Embedding(vocab_size, 100)
        pretrained_weight = np.array(pretrained)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
    def forward(self, x, aspects): 
        x = self.embeddings(x)#batch,50,100
        #va = self.embeddings(aspects[0])
        #t_va = va.unsqueeze(0).expand(x.shape[0], x.shape[1], 100)
        output, (hn, cn) =  self.bilstm(x.float())#batch,50,200
        H, hN = output, output[:,-1,:]
        #con = torch.cat((t_va, H), 2)
        #t1 = F.max_pool2d(F.relu(con), (1, 4))
        #con = t1.view(x.size()[0], -1)
        #con = F.relu(self.fc1(con))
        #alpha = con.unsqueeze(1)
        #rr = torch.matmul(alpha,H).squeeze(1)
        #yy = torch.cat((F.relu(self.fc2(rr)), F.relu(self.fc3(hN))), 1)
        #x = self.fc4(yy)
        x  = self.fctest(hN)
        return x
    

net = Net(pretrained, len(pretrained))
print(net)
from torch import optim
criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)
'''
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, pretrained, vocab_size):
        super(Net, self).__init__()
        #self.rnn1 = nn.RNN(100, 100) 
        self.bilstm = nn.LSTM(input_size = 100, hidden_size = 100, bidirectional = True)
        #self.fc1   = nn.Linear(1250, 120) 
        self.fch = nn.Linear(200, 100)
        self.fcv = nn.Linear(200, 100)
        self.fcw = nn.Linear(200, 1)
        self.fcwp = nn.Linear(200,50)
        self.fcwh = nn.Linear(200,50)
        self.fcwsen = nn.Linear(200, 50)
        self.fc1   = nn.Linear(3750, 120) 
        self.fc2   = nn.Linear(120, 50)
        self.fc3   = nn.Linear(50, 2)
        self.embeddings = nn.Embedding(vocab_size, 100)
        pretrained_weight = np.array(pretrained)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
    def forward(self, mode, x, x_sent, aspects): 
        x = self.embeddings(x) #[4,50,100]
        bn = nn.BatchNorm1d(50)
        x = bn(x)
        batch_size = x.shape[0]
        N = x.shape[1]
        output, (hn, cn) =  self.bilstm(x.float())  #[4,50,200]
        hN = output[:,-1,:]
        va = self.embeddings(aspects[0]) #[1, 100]
        t_va = torch.matmul(torch.ones((x.shape[0],50, 1)), va)#[4, 50, 100]
        x_sent = x_sent.float()
        #[4, 50] *? = [4, 50, 100]
        sent_va = torch.mul(torch.matmul(x_sent.reshape((x.shape[0], 50, 1)), torch.ones((x.shape[0],1, 100))), t_va)  #[4, 50, 100]
        t_va = torch.cat((sent_va, t_va),  2)
        t_va = t_va.reshape((-1, 200)) #[4*50, 100]
        x = output.reshape((-1, 200)) #[4*50, 200]
        x = torch.tanh(torch.cat((self.fch(x), self.fcv(t_va)), 1))
        M = x.reshape((batch_size, N, -1)) #[4, 50, 200]
        alpha = F.softmax(self.fcw(M)).reshape((batch_size, 1, -1))#[4,50]
        rr = torch.matmul(alpha, output).reshape((batch_size, -1)) #[4, 200]
        x_sent = x_sent.reshape((batch_size, 1, -1))
        ss = torch.matmul(x_sent.float(), output).reshape((batch_size, -1))
        final = torch.tanh(self.fcwh(hN) + self.fcwp(rr) + self.fcwsen(ss))
        x = self.fc3(final)   
        return x, alpha
net = Net(pretrained, len(pretrained))    
from torch import optim
criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
optimizer = optim.Adagrad(net.parameters(), lr=0.01)#, momentum=0.9)

In [39]:
import torch.nn as nn
import torch.nn.functional as F
result_set1 = {}
for key in sub_set:
    if key != '配置':
        continue
    print(key)
    res = []
    with torch.no_grad():
        correct, total = 0, 0
        testset = [(han_x_test[i],np.array(x_test_sent_vecs[key][i])) for i in range(len(han_x_test))]
        testloader = torch.utils.data.DataLoader(
                            testset, 
                            batch_size=4,
                            shuffle=False, 
                            num_workers=2)
        for data in testloader:
            # 梯度清零
            optimizer.zero_grad()
            # forward + backward 
            (inputs, sentis) = data
            va = [[word2ind[key]]]
            va = torch.from_numpy(np.array(va))
            net = torch.load('./../2classtrain/net' + key + '.pkl')
            outputs,alpha = net('test', inputs, sentis, va)
            _, predicted = torch.max(outputs, 1)
            res = res + [int(predicted[i]) for i in range(len(predicted))]
    real_text = []
    for i in range(len(han_x_test)):
        temp = []
        for j in range(len(han_x_test[i])):
            temp.append(ind2word[han_x_test[i][j]])
        real_text.append(''.join(temp))
    for zz in range(len(res)):
        if res[zz] == 0:
            set_ = ['音质','配置','发动机','四驱','导航','屏幕','中控','音响']
            for k in set_:
                if k in real_text[zz]:
                    res[zz] = 1
                    break
    result_set1[key] = res


配置


/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:80: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [40]:
len(result_set1['配置'])

2364

In [41]:

'''
class Net(nn.Module):
    def __init__(self, pretrained, vocab_size):
        super(Net, self).__init__()
        self.bilstm = nn.LSTM(input_size = 100, hidden_size = 100, bidirectional = True) 
        #self.fc1   = nn.Linear(3750, 50) 
        #self.fc2   = nn.Linear(200, 50)
        #self.fc3   = nn.Linear(200, 50)
        #self.fc4   = nn.Linear(100, 3)
        self.fctest   = nn.Linear(200, 3)
        self.embeddings = nn.Embedding(vocab_size, 100)
        pretrained_weight = np.array(pretrained)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
    def forward(self, x, aspects): 
        x = self.embeddings(x)#batch,50,100
        #va = self.embeddings(aspects[0])
        #t_va = va.unsqueeze(0).expand(x.shape[0], x.shape[1], 100)
        output, (hn, cn) =  self.bilstm(x.float())#batch,50,200
        H, hN = output, output[:,-1,:]
        #con = torch.cat((t_va, H), 2)
        #t1 = F.max_pool2d(F.relu(con), (1, 4))
        #con = t1.view(x.size()[0], -1)
        #con = F.relu(self.fc1(con))
        #alpha = con.unsqueeze(1)
        #rr = torch.matmul(alpha,H).squeeze(1)
        #yy = torch.cat((F.relu(self.fc2(rr)), F.relu(self.fc3(hN))), 1)
        #x = self.fc4(yy)
        x  = self.fctest(hN)
        return x
    

net = Net(pretrained, len(pretrained))
print(net)
from torch import optim
criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)
'''
import torch.nn as nn
import torch.nn.functional as F

class sNet(nn.Module):
    def __init__(self, pretrained, vocab_size):
        super(sNet, self).__init__()
        #self.rnn1 = nn.RNN(100, 100) 
        self.bilstm = nn.LSTM(input_size = 100, hidden_size = 100, bidirectional = True)
        #self.fc1   = nn.Linear(1250, 120) 
        self.fch = nn.Linear(200, 100)
        self.fcv = nn.Linear(200, 100)
        self.fcw = nn.Linear(200, 1)
        self.fcwp = nn.Linear(200,50)
        self.fcwh = nn.Linear(200,50)
        self.fcwsen = nn.Linear(200, 50)
        self.fc1   = nn.Linear(3750, 120) 
        self.fc2   = nn.Linear(120, 50)
        self.fc3   = nn.Linear(50, 3)
        self.embeddings = nn.Embedding(vocab_size, 100)
        pretrained_weight = np.array(pretrained)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
    def forward(self, mode, x, x_sent, aspects): 
        x = self.embeddings(x) #[4,50,100]
        if mode == 'train':
            drop = nn.Dropout(0.5)
            x = drop(x)
        bn = nn.BatchNorm1d(50)
        x = bn(x)
        batch_size = x.shape[0]
        N = x.shape[1]
        output, (hn, cn) =  self.bilstm(x.float())  #[4,50,200]
        hN = output[:,-1,:]
        va = self.embeddings(aspects[0]) #[1, 100]
        t_va = torch.matmul(torch.ones((x.shape[0],50, 1)), va)#[4, 50, 100]
        x_sent = x_sent.float()
        #[4, 50] *? = [4, 50, 100]
        sent_va = torch.mul(torch.matmul(x_sent.reshape((x.shape[0], 50, 1)), torch.ones((x.shape[0],1, 100))), t_va)  #[4, 50, 100]
        t_va = torch.cat((sent_va, t_va),  2)
        t_va = t_va.reshape((-1, 200)) #[4*50, 100]
        x = output.reshape((-1, 200)) #[4*50, 200]
        x = torch.tanh(torch.cat((self.fch(x), self.fcv(t_va)), 1))
        M = x.reshape((batch_size, N, -1)) #[4, 50, 200]
        alpha = F.softmax(self.fcw(M)).reshape((batch_size, 1, -1))#[4,50]
        rr = torch.matmul(alpha, output).reshape((batch_size, -1)) #[4, 200]
        x_sent = x_sent.reshape((batch_size, 1, -1))
        ss = torch.matmul(x_sent.float(), output).reshape((batch_size, -1))
        final = torch.tanh(self.fcwh(hN) + self.fcwp(rr) + self.fcwsen(ss))
        x = self.fc3(final)   
        return x, alpha
    


In [42]:
ii = 0
import time
result_set2 = {}
for key in sub_set:
    if key !='配置':
        continue
    if ii == 10:
        break
    ii += 1
    net = sNet(pretrained, len(pretrained))
    cur_max_acc = 0.0
    print(net)
    from torch import optim
    with torch.no_grad():
        res = []
        correct, total = 0, 0
        testset = [(han_x_test[i],np.array(x_test_sent_vecs[key][i])) for i in range(len(han_x_test))]
        testloader = torch.utils.data.DataLoader(
                            testset, 
                            batch_size=4,
                            shuffle=False, 
                            num_workers=2)
        for data in testloader:
            images, sentis = data
            va = [[word2ind[key]]]
            va = torch.from_numpy(np.array(va))
            net = torch.load('./../2classtrain/2net' + key + '.pkl')
            outputs, alpha = net('test', images,sentis, va)
            _, predicted = torch.max(outputs, 1)
            res = res + [int(predicted[i]) for i in range(len(predicted))]
    real_text = []
    for i in range(len(han_x_test)):
        temp = []
        for j in range(len(han_x_test[i])):
            temp.append(ind2word[han_x_test[i][j]])
        real_text.append(''.join(temp))
    for zz in range(len(res)):
        set_ = {'配置高','音质好','顶配','配置新','配置好','不错','顶级'}
        for k in set_:
            if k in real_text[zz]:
                res[zz] = 2
                #print(real_text[zz].split('null')[0], res[zz], ll[zz])
                break
        set_ = {'配置低','低配','垃圾导航','鸡肋','音质不','垃圾中控','中控垃圾'}
        for k in set_:
            if k in real_text[zz]:
                res[zz] = 0
                #print(real_text[zz].split('null')[0], res[zz], ll[zz])
                break
    result_set2[key] = res


sNet(
  (bilstm): LSTM(100, 100, bidirectional=True)
  (fch): Linear(in_features=200, out_features=100, bias=True)
  (fcv): Linear(in_features=200, out_features=100, bias=True)
  (fcw): Linear(in_features=200, out_features=1, bias=True)
  (fcwp): Linear(in_features=200, out_features=50, bias=True)
  (fcwh): Linear(in_features=200, out_features=50, bias=True)
  (fcwsen): Linear(in_features=200, out_features=50, bias=True)
  (fc1): Linear(in_features=3750, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=3, bias=True)
  (embeddings): Embedding(19307, 100)
)


/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:80: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [43]:
len(result_set1['配置']),len(result_set2['配置'])

(2364, 2364)

In [44]:
c_ids = list(test_data['content_id'])

In [45]:
result_dic = {}
count=0
for i in range(len(result_set1['配置'])):
    if result_set1['配置'][i] == 1:
        if result_set2['配置'][i] - 1 != 0:
            count += 1
        result_dic[c_ids[i]] = result_set2['配置'][i] - 1
import pickle
pickle.dump(result_dic, open('./split_result/配置.txt', 'wb'))
len(result_dic), count

(588, 55)

In [46]:
result_dic

{'BeSIAUzoTWH9muYl': 0,
 'SXYTBWg6FxuboMnd': 0,
 'RDgTmzW9xBfY6pGv': -1,
 '8cMwzsxkL4XtuDG1': 0,
 'MOUrvjDsIJpbw6dQ': 0,
 't01PVoDSd9Lsy5HN': 0,
 '1rGSYnOx9Fb2ULQX': 0,
 'Q4lp0HBNfTD2xYAv': 0,
 'M3Hqx1Un8ygzFTtl': 0,
 'cIevXAM5FRp14Nni': 0,
 '8jCcfemUlPaQFx67': 0,
 'EKI7PjFs81Lc52uS': 0,
 '5b2XhIBYPOCfMzcu': 0,
 'QRw7ngTJ2A5CYBGW': 0,
 '06vPIQBqVNe2dl3f': 0,
 'ARIng69ZHsY7DCN1': 0,
 'S1WzLCPqeIT8sMmZ': 0,
 'MSn1hW8XCsGTkpxY': 0,
 '14dQnL9lxMYeksH3': 1,
 'Wf9T283SZphL6tQ4': 0,
 'D2HpBfqwoecTkhdS': 0,
 'dm1W2BIEyJ0Gsxp8': 0,
 'Mglj9pKWkY7Isixm': 0,
 'otT6DmhdEpx5Q74R': 0,
 'QG07en46S5qxWUud': 0,
 '5jHabez9oxP4CY8O': 0,
 'qpPHjUw7IMJNFSub': 1,
 'a52KoCrScpnW8xlR': 1,
 '4PUWD5HkGZFdcYr6': 0,
 '4SDY29tXIivB6VLZ': 0,
 'JDwFsuZQMy5mVRzp': 0,
 'dm6eXoIg9Df0nFaV': 0,
 '3GtNOfDcna4Mzk5i': 0,
 '3s9NFKhCQk70Ti2E': 0,
 'V3xslWKOyv9ILquX': 0,
 'Krp3Gzk2aNhHJ6R7': 0,
 'gnpKm9FPl4Y3OL8h': 0,
 'PXdW23e95gLOhBaf': 0,
 'TBy1z62utW8GcvIs': 0,
 'sJlZXtVAdKeq0gha': 1,
 'bHtilDz4Yf8v7TmS': 0,
 'YUG1zybnqjSDm